In [9]:

import pandas as pd
def map_forward_backward_month(s="2020-01-09", method="backward"):
    def f(year_month):
        y, m = year_month.split("-")
        if m == "01":
            m = "12"
            y = str(int(y) - 1)
        else:
            m = str(int(m) - 1).zfill(2)
        return y + "-" + m
    md_dict = {
        "/2": "28",
        "01": "31",
        "02": "28",
        "03": "31",
        "04": "30",
        "05": "31",
        "06": "30",
        "07": "31",
        "08": "31",
        "09": "30",
        "10": "31",
        "11": "30",
        "12": "31"}
    if method == "forward":
        return s[:8] + md_dict[s[5:7]]
    else:
        return f(s[:7]) + "-" + md_dict[f(s[:7])[5:]] 


class GenerateDataSet:
    
    def __init__(self, marketing=None, socio=None):
        self.marketing = marketing 
        self.socio = socio 
 
    def _normalize_data(self, method="forward"):
        self.marketing["DATE"] = self.marketing["DATE"].apply(lambda date: map_forward_backward_month(date, method=method))
        return self.marketing.join(self.socio.set_index("DATE"), on="DATE")

    def create_data(self, method="forward"):
        return self._normalize_data(method=method).drop(["DATE", "DURATION_CONTACT"] , axis=1)
    


# read the data from sql ... enfin dans la vraie vie 
socio = pd.read_csv("/Users/heps/Downloads/sc.csv", sep=",")
marketing = pd.read_csv("/Users/heps/Downloads/marketing.csv", sep=",")

# merge 
instance = GenerateDataSet(socio=socio, marketing=marketing).create_data()


# see the output


instance.head(1).to_dict()

{'AGE': {0: 58},
 'JOB_TYPE': {0: 'Manager'},
 'STATUS': {0: 'Marié'},
 'EDUCATION': {0: 'Tertiaire'},
 'HAS_DEFAULT': {0: 'No'},
 'BALANCE': {0: 2143},
 'HAS_HOUSING_LOAN': {0: 'Yes'},
 'HAS_PERSO_LOAN': {0: 'No'},
 'CONTACT': {0: nan},
 'NB_CONTACT': {0: 1},
 'NB_DAY_LAST_CONTACT': {0: -1},
 'NB_CONTACT_LAST_CAMPAIGN': {0: 0},
 'RESULT_LAST_CAMPAIGN': {0: nan},
 'SUBSCRIPTION': {0: 'No'},
 'EMPLOYMENT_VARIATION_RATE': {0: 1.1},
 'IDX_CONSUMER_PRICE': {0: 93.994},
 'IDX_CONSUMER_CONFIDENCE': {0: -36.4},
 'NB_EMPLOYE': {0: 5191.0}}

In [7]:
a = pd.DataFrame.from_dict({'DATE': {0: '2008-05-05'},
 'AGE': {0: 58},
 'JOB_TYPE': {0: 'Manager'},
 'STATUS': {0: 'Marié'},
 'EDUCATION': {0: 'Tertiaire'},
 'HAS_DEFAULT': {0: 'No'},
 'BALANCE': {0: 2143},
 'HAS_HOUSING_LOAN': {0: 'Yes'},
 'HAS_PERSO_LOAN': {0: 'No'},
 'CONTACT': {0: 'nan'},
 'DURATION_CONTACT': {0: 261},
 'NB_CONTACT': {0: 1},
 'NB_DAY_LAST_CONTACT': {0: -1},
 'NB_CONTACT_LAST_CAMPAIGN': {0: 0},
 'RESULT_LAST_CAMPAIGN': {0: 'nan'},
 'SUBSCRIPTION': {0: 'No'}})

In [8]:
a

,DATE,AGE,JOB_TYPE,STATUS,EDUCATION,HAS_DEFAULT,BALANCE,HAS_HOUSING_LOAN,HAS_PERSO_LOAN,CONTACT,DURATION_CONTACT,NB_CONTACT,NB_DAY_LAST_CONTACT,NB_CONTACT_LAST_CAMPAIGN,RESULT_LAST_CAMPAIGN,SUBSCRIPTION
0,2008-05-05,58,Manager,Marié,Tertiaire,No,2143,Yes,No,nan,261,1,-1,0,nan,No


In [35]:
{"DATE": "2008-05-05",
 "AGE": 58,
 "JOB_TYPE": 'Manager',
 'STATUS': 'Marié',
 'EDUCATION': 'Tertiaire',
 'HAS_DEFAULT': 'No',
 'BALANCE': 2143,
 'HAS_HOUSING_LOAN': 'Yes',
 'HAS_PERSO_LOAN': 'No',
 'CONTACT': nan,
 'DURATION_CONTACT': 261,
 'NB_CONTACT': 1,
 'NB_DAY_LAST_CONTACT': -1,
 'NB_CONTACT_LAST_CAMPAIGN': 0,
 'RESULT_LAST_CAMPAIGN': nan}

NameError: name 'nan' is not defined

# pipeline

In [19]:
# categorical pipeline 
#instance.info()
features, target = instance.drop("SUBSCRIPTION", axis=1), instance.SUBSCRIPTION
features.shape, target.shape

((45136, 17), (45136,))

In [21]:
pip install sklearn 

Processing /Users/heps/Library/Caches/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e/sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-0.23.1-cp37-cp37m-macosx_10_9_x86_64.whl (7.2 MB)
     |████████████████████████████████| 28.7 MB 4.1 MB/s eta 0:00:01    |██████████▊                     | 9.6 MB 5.3 MB/s eta 0:00:04     |███████████████████████▉        | 21.4 MB 4.1 MB/s eta 0:00:02
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)
Note: you may need to restart the kernel to use updated packages.


In [23]:
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

# type of  variables
categorical_features = ["JOB_TYPE", "STATUS", "EDUCATION", "HAS_DEFAULT", "HAS_HOUSING_LOAN", "HAS_PERSO_LOAN", "CONTACT", "RESULT_LAST_CAMPAIGN"]
#instance[categorical_variable]

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

numeric_features = ["AGE", "BALANCE", "NB_CONTACT", "NB_DAY_LAST_CONTACT", "NB_CONTACT_LAST_CAMPAIGN", "EMPLOYMENT_VARIATION_RATE", "IDX_CONSUMER_PRICE","IDX_CONSUMER_CONFIDENCE", "NB_EMPLOYE"]
#instance[numeric_features]
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

clf.fit(features, target)

/Users/heps/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['AGE', 'BALANCE',
                                                   'NB_CONTACT',
                                                   'NB_DAY_LAST_CONTACT',
                                                   'NB_CONTACT_LAST_CAMPAIGN',
                                                   'EMPLOYMENT_VARIATION_RATE',
                                                   'IDX_CONSUMER_PRICE',
                                                   'IDX_CONSUMER_CONFIDENCE',
                                               

In [25]:
client_data = {}


TypeError: score() missing 1 required positional argument: 'X'

In [26]:
from sklearn import set_config
set_config(display='diagram')
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['AGE', 'BALANCE',
                                                   'NB_CONTACT',
                                                   'NB_DAY_LAST_CONTACT',
                                                   'NB_CONTACT_LAST_CAMPAIGN',
                                                   'EMPLOYMENT_VARIATION_RATE',
                                                   'IDX_CONSUMER_PRICE',
                                                   'IDX_CONSUMER_CONFIDENCE',
                                                   'NB_EMPLOYE']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['JOB_TYPE', 'STATUS',
                                                   'EDUCATION', 'HAS_DEFAULT',
                                                   'HAS_HOUSING_LOAN',
                                                   'HAS_PERSO_LOAN', 'CONTACT',
                                                   'RESULT_LAST_CAMPAIGN'])])),
                ('classifier', LogisticRegression())])

In [36]:
import json

class Object:
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=True, indent=4)

In [37]:
me = Object()
me.name = "Onur"
me.age = 35
me.dog = Object()
me.dog.name = "Apollo"

In [43]:




pd.DataFrame.from_dict({'DATE': '2008-05-05', 'AGE': 58, 'JOB_TYPE': 'Manager', 'STATUS': 'Mari\u00e9', 'EDUCATION': 'Tertiaire', 'HAS_DEFAULT': 'No', 'BALANCE': 2143, 'HAS_HOUSING_LOAN': 'Yes', 'HAS_PERSO_LOAN': 'No', 'CONTACT': 'nan', 'DURATION_CONTACT': 261, 'NB_CONTACT': 1, 'NB_DAY_LAST_CONTACT': -1, 'NB_CONTACT_LAST_CAMPAIGN': 0, 'RESULT_LAST_CAMPAIGN': 'nan'}, orient="index").T








,DATE,AGE,JOB_TYPE,STATUS,EDUCATION,HAS_DEFAULT,BALANCE,HAS_HOUSING_LOAN,HAS_PERSO_LOAN,CONTACT,DURATION_CONTACT,NB_CONTACT,NB_DAY_LAST_CONTACT,NB_CONTACT_LAST_CAMPAIGN,RESULT_LAST_CAMPAIGN
0,2008-05-05,58,Manager,Marié,Tertiaire,No,2143,Yes,No,nan,261,1,-1,0,nan
